## 4.2 K近傍法(KNN)

In [ ]:
# 4.2 KNN
import polars as pl
from func import autoscaling, delete_zero_std_columns, load_data
from sklearn.neighbors import NearestNeighbors  # k-NN

k_in_knn = 5  # k-NN における k

# トレーニングデータの読み込み
x_train, index_train = load_data("../test_data/resin.csv", index="")
y_train = x_train.get_column("property")  # 目的変数
x_train = x_train.drop(y_train.name)  # 説明変数
# 予測用データの読み込み
x_pred, index_pred = load_data("../test_data/resin_prediction.csv", index="")

# 標準偏差が 0 の特徴量の削除
x_train = delete_zero_std_columns(x_train)
x_pred = x_pred.select(x_train.columns)

# オートスケーリング
autoscaled_x_train = autoscaling(x_train)
autoscaled_x_pred = autoscaling(x_pred, x_train)

# k-NN による AD(Applicability Domain: モデルの適用範囲) の構築
"""
ADとは、予測モデルが信頼できる範囲を示す概念.モデルの内挿・外挿とは一致しない場合がある.
AD内のサンプルに対する予測は信頼性が高いとされる.
内挿: Xの範囲内での予測
外挿: Xの範囲外での予測
AD: 単純に範囲のみでなく、ほかの方法で定義される場合もある.
k-NN による AD では、各サンプルに対して k 個の最近傍サンプルとの距離を計算し、
その平均距離が一定の閾値以下であれば AD 内、閾値以上であれば AD 外と判定する。
"""
# k-NN による AD では、トレーニングデータ x を ad_model に格納することに対応
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric="euclidean")
ad_model.fit(autoscaled_x_train)

# knn距離とインデックスが戻り値
# k_in_knn + 1(自分自身)個を抽出
knn_distance_train, _ = ad_model.kneighbors(autoscaled_x_train, k_in_knn + 1)
# 自分自身との距離を削除
knn_distance_train_wo_self = pl.DataFrame(knn_distance_train).drop(pl.first())
# 自分以外のサンプルとの距離の平均
mean_of_knn_distance_train = knn_distance_train_wo_self.mean_horizontal().rename(
    "mean_of_knn_distance"
)
# トレーニングデータに対して、AD の中/外を判定しcsv出力(閾値はデータの96%)
ad_threshold = mean_of_knn_distance_train.quantile(0.96, "lower")
pl.DataFrame(
    {
        "": index_train,
        "mean_of_knn_distance": mean_of_knn_distance_train,
        "within_ad": mean_of_knn_distance_train.le(ad_threshold),
    }
).write_csv("../output/04_02/mean_of_knn_distance_train.csv", quote_style="never")

# 予測用データに対して同様の処理を行う
knn_distance_pred, knn_index_pred = ad_model.kneighbors(autoscaled_x_pred)
# k_in_knn 個の距離の平均
mean_of_knn_distance_pred = (
    pl.DataFrame(knn_distance_pred).mean_horizontal().rename("mean_of_knn_distance")
)
pl.DataFrame(
    {
        "": index_pred,
        "mean_of_knn_distance": mean_of_knn_distance_pred,
        "within_ad": mean_of_knn_distance_pred.le(ad_threshold),
    }
).write_csv("../output/04_02/mean_of_knn_distance_prediction.csv", quote_style="never")

## 4.2 OCSVM (One-Class Support Vector Machine)

In [ ]:
# 4-2 One-Class Support Vector Machine

import polars as pl
from func import autoscaling, delete_zero_std_columns, load_data
from sklearn.svm import OneClassSVM

# OCSVM における nu。
# トレーニングデータにおけるサンプル数に対する、サポートベクターの数の下限の割合
ocsvm_nu = 0.04
ocsvm_gamma = 0.1  # OCSVM における gamma

x_train, index_train = load_data("../test_data/resin.csv", index="")
x_pred, index_pred = load_data("../test_data/resin_prediction.csv", index="")

# データ分割
y_train = x_train.get_column("property")  # 目的変数
x_train = x_train.drop(y_train.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
x_train = delete_zero_std_columns(x_train)
x_pred = x_pred.select(x_train.columns)

# オートスケーリング
autoscaled_x_train = autoscaling(x_train)
autoscaled_x_pred = autoscaling(x_pred, x_train)

# OCSVM による ADの決定
ad_model = OneClassSVM(kernel="rbf", gamma=ocsvm_gamma, nu=ocsvm_nu)  # AD モデルの宣言
ad_model.fit(autoscaled_x_train)  # モデル構築

# トレーニングデータのデータ密度 (f(x) の値)を取得し、
# ADの中/外を判定(0以上:AD内、0未満:AD外)
data_density_train = pl.Series(ad_model.decision_function(autoscaled_x_train))
pl.DataFrame(
    {
        "": index_train,
        "ocsvm_data_density": data_density_train,
        "within_ad_flag": data_density_train.ge(0),
    }
).write_csv("../output/04_02/ocsvm_data_density_train.csv", quote_style="never")

number_of_outliers_in_training_data = data_density_train.lt(0).sum()
support_vector_ratio = ad_model.support_.size / x_train.height
outlier_ratio_train = number_of_outliers_in_training_data / x_train.height
print(
    f"\nトレーニングデータにおけるサポートベクター数 :{ad_model.support_.size}"
    f"\nトレーニングデータにおけるサポートベクターの割合 :{support_vector_ratio}"
    f"\nトレーニングデータにおける外れサンプル数 :{number_of_outliers_in_training_data}"
    f"\nトレーニングデータにおける外れサンプルの割合 :{outlier_ratio_train}"
)

# 予測用データセットも同様に処理
data_density_prediction = pl.Series(ad_model.decision_function(autoscaled_x_pred))
pl.DataFrame(
    {
        "": index_pred,
        "ocsvm_data_density": data_density_prediction,
        "within_ad_flag": data_density_prediction.ge(0),
    }
).write_csv("../output/04_02/ocsvm_data_density_prediction.csv", quote_style="never")

num_of_outliers_in_pred_data = data_density_prediction.lt(0).sum()
outlier_ratio_pred = num_of_outliers_in_pred_data / x_pred.height
print(
    f"\n予測用データセットにおける外れサンプル数 :{num_of_outliers_in_pred_data}",
    f"\n予測用データセットにおける外れサンプルの割合 :{outlier_ratio_pred}",
)


トレーニングデータにおけるサポートベクター数 :10
トレーニングデータにおけるサポートベクターの割合 :0.5
トレーニングデータにおける外れサンプル数 :4
トレーニングデータにおける外れサンプルの割合 :0.2

予測用データセットにおける外れサンプル数 :4757 
予測用データセットにおける外れサンプルの割合 :0.5826800587947085


## 4.2 OCSVM (One-Class Support Vector Machine) Gamma optimization

In [ ]:
# 4.2 One-Class Support Vector Machine Gamma optimization

# 4-2 One-Class Support Vector Machine
import polars as pl
from func import autoscaling, calc_optimal_gamma, delete_zero_std_columns, load_data
from sklearn.svm import OneClassSVM

x_train, index_train = load_data("../test_data/resin.csv", index="")
x_pred, index_pred = load_data("../test_data/resin_prediction.csv", index="")

# データ分割
y_train = x_train.get_column("property")  # 目的変数
x_train = x_train.drop(y_train.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
x_train = delete_zero_std_columns(x_train)
x_pred = x_pred.select(x_train.columns)

# オートスケーリング
autoscaled_x_train = autoscaling(x_train)
autoscaled_x_pred = autoscaling(x_pred, x_train)

# グラム行列の分散最大化によるガウシアンカーネルのγの最適化
ocsvm_gammas = 2.0 ** pl.arange(-20, 11, eager=True)  # OCSVM における gamma の候補
optimal_gamma = calc_optimal_gamma(autoscaled_x_train, ocsvm_gammas)
print("最適化された gamma :", optimal_gamma)

# OCSVM による AD
# OCSVM における nu。
# トレーニングデータにおけるサンプル数に対する、サポートベクターの数の下限の割合
ocsvm_nu = 0.04

# AD モデルの宣言
ad_model = OneClassSVM(kernel="rbf", gamma=optimal_gamma, nu=ocsvm_nu)
ad_model.fit(autoscaled_x_train)  # モデル構築

# トレーニングデータのデータ密度 (f(x) の値)を取得し、
# ADの中/外を判定(0以上:AD内、0未満:AD外)
data_density_train = pl.Series(ad_model.decision_function(autoscaled_x_train))
pl.DataFrame(
    {
        "": index_train,
        "ocsvm_data_density": data_density_train,
        "within_ad_flag": data_density_train.ge(0),
    }
).write_csv(
    "../output/04_02/ocsvm_gamma_optimization_data_density_train.csv",
    quote_style="never",
)

number_of_outliers_in_training_data = data_density_train.lt(0).sum()
support_vector_ratio = ad_model.support_.size / x_train.height
outlier_ratio_train = number_of_outliers_in_training_data / x_train.height
print(
    f"\nトレーニングデータにおけるサポートベクター数 :{ad_model.support_.size}"
    f"\nトレーニングデータにおけるサポートベクターの割合 :{support_vector_ratio}"
    f"\nトレーニングデータにおける外れサンプル数 :{number_of_outliers_in_training_data}"
    f"\nトレーニングデータにおける外れサンプルの割合 :{outlier_ratio_train}"
)

# 予測用データセットも同様に処理
data_density_prediction = pl.Series(ad_model.decision_function(autoscaled_x_pred))
pl.DataFrame(
    {
        "": index_pred,
        "ocsvm_data_density": data_density_prediction,
        "within_ad_flag": data_density_prediction.ge(0),
    }
).write_csv(
    "../output/04_02/ocsvm_gamma_optimization_data_density_prediction.csv",
    quote_style="never",
)

num_of_outliers_in_pred_data = data_density_prediction.lt(0).sum()
outlier_ratio_pred = num_of_outliers_in_pred_data / x_pred.height
print(
    f"\n予測用データセットにおける外れサンプル数 :{num_of_outliers_in_pred_data}",
    f"\n予測用データセットにおける外れサンプルの割合 :{outlier_ratio_pred}",
)

最適化された gamma : 0.25

トレーニングデータにおけるサポートベクター数 :14
トレーニングデータにおけるサポートベクターの割合 :0.7
トレーニングデータにおける外れサンプル数 :6
トレーニングデータにおける外れサンプルの割合 :0.3

予測用データセットにおける外れサンプル数 :6252 
予測用データセットにおける外れサンプルの割合 :0.7658010779029887


## 4.3 アンサンブル学習

In [ ]:
# 4.3 アンサンブル学習

import pickle
from pathlib import Path

import polars as pl
from func import (
    autoscaling,
    delete_zero_std_columns,
    load_data,
    optimize_hyperparameters_by_cv,
    rescaling,
)
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from tqdm.notebook import tqdm

svr_cs = 2.0 ** pl.arange(-5, 11, eager=True)  # C の候補
svr_epsilons = 2.0 ** pl.arange(-10, 1, eager=True)  # epsilon の候補
svr_gammas = 2.0 ** pl.arange(-20, 11, eager=True)  # gamma の候補

x_train, index_train = load_data("../test_data/resin.csv", index="")
x_pred, index_pred = load_data("../test_data/resin_prediction.csv", index="")

# データ分割
y_train = x_train.get_column("property")  # 目的変数
x_train = x_train.drop(y_train.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
x_train = delete_zero_std_columns(x_train)
x_pred = x_pred.select(x_train.columns)

# オートスケーリング
autoscaled_x_train = autoscaling(x_train)
autoscaled_x_pred = autoscaling(x_pred, x_train)
autoscaled_y = autoscaling(y_train)

# 各サブデータセットで選択される説明変数の数の割合を設定。0 < x < 1
rate_of_selected_x_variables = 0.75
print("各サブデータセットにおける説明変数の割合 :", rate_of_selected_x_variables)

num_of_sub_datasets = 30  # サブデータセットの数
selected_autoscaled_x_trains = [
    autoscaled_x_train.select(
        pl.Series("x_column", autoscaled_x_train.columns)
        .sample(fraction=rate_of_selected_x_variables, seed=seed, shuffle=False)
        .to_list()
    )
    for seed in range(num_of_sub_datasets)
]
selected_x_cols_list = [
    selected_autoscaled_x_train.columns
    for selected_autoscaled_x_train in selected_autoscaled_x_trains
]

# クロスバリデーションの分割の設定
fold_number = 10  # N-fold CV の N
cv = KFold(n_splits=fold_number, random_state=9, shuffle=True)
# サブデータセットごとにモデル構築
sub_model_list = []
for selected_autoscaled_x_train in tqdm(selected_autoscaled_x_trains):
    # ハイパーパラメータC, ε, γの最適化
    (
        optimal_c,
        optimal_epsilon,
        optimal_gamma,
    ) = optimize_hyperparameters_by_cv(
        SVR,
        autoscaled_x_train,
        autoscaled_y,
        y_train,
        svr_cs,
        svr_epsilons,
        svr_gammas,
        cv,
        verbose=False,
    )
    sub_model = SVR(
        kernel="rbf",
        C=optimal_c,
        epsilon=optimal_epsilon,
        gamma=optimal_gamma,
    ).fit(selected_autoscaled_x_train, autoscaled_y)
    sub_model_list.append(sub_model)


# サブデータセットの説明変数の種類やサブデータセットを用いて構築されたモデルを保存
selected_bin_path = Path("../output/04_03/selected_x_variables_svr_gaussian.bin")
sub_models_bin_path = Path("../output/04_03/sub_models_svr_gaussian.bin")
with selected_bin_path.open("wb") as f:
    pickle.dump(selected_x_cols_list, f)
with sub_models_bin_path.open("wb") as f:
    pickle.dump(sub_model_list, f)

# サブデータセットの説明変数の種類やサブデータセットを用いて構築されたモデルを読み込み
with selected_bin_path.open("rb") as f:
    selected_x_cols_list: list[pl.Series] = pickle.load(f)  # noqa: S301
with sub_models_bin_path.open("rb") as f:
    sub_model_list: list[SVR] = pickle.load(f)  # noqa: S301

estimated_y_pred_new = pl.DataFrame()
for i, (sub_model, selected_x_cols) in enumerate(
    tqdm(
        zip(sub_model_list, selected_x_cols_list, strict=True),
        total=len(sub_model_list),
    )
):
    # 説明変数の選択
    selected_autoscaled_x_prediction = autoscaled_x_pred.select(selected_x_cols)
    # 予測用データセットの y の推定
    estimated_y_pred = pl.Series(sub_model.predict(selected_autoscaled_x_prediction))
    # スケールをもとに戻します
    estimated_y_pred_new = estimated_y_pred_new.with_columns(
        rescaling(estimated_y_pred, y_train).rename(f"{i}")
    )

# 予測用データセットの推定値の平均値
estimated_y_pred_new.select(
    index_pred.alias(""),
    estimated_y_pred_new.mean_horizontal().alias("mean_of_estimated_y"),
    # std_horizontal()がないので代替処理
    pl.concat_list(pl.all()).list.std().alias("std_of_estimated_y"),
).write_csv(
    "../output/04_03/estimated_y_prediction_ensemble_svr_gaussian.csv",
    quote_style="never",
)

各サブデータセットにおける説明変数の割合 : 0.75


  0%|          | 0/30 [00:00<?, ?it/s]